In [4]:
import os

import numpy as np
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import ast

In [5]:
descriptions = pd.read_csv(os.path.join('data', 'descriptions.csv')).rename(columns={'movie': 'item'})

In [14]:
descriptions.haad

AttributeError: 'DataFrame' object has no attribute 'haad'

In [138]:
def make_item_descriptions(max_sentence_length=100):
    descriptions = pd.read_csv(os.path.join('data', 'descriptions.csv')).rename(columns={'movie': 'item'})
    descriptions['crew'] = descriptions['crew'].apply(lambda x: ast.literal_eval(x))
    descriptions['cast'] = descriptions['cast'].apply(lambda x: ast.literal_eval(x))
    texts = descriptions.description
    texts = texts.apply(lambda x: x.strip().split())
    descriptions['description'] = [['crew'] + crew + ['cast'] + cast + ['text'] + text
                                   for crew, cast, text in zip(descriptions.crew, descriptions.cast, texts)]
    texts = descriptions.description
    dictionary = Dictionary(texts.values)
    eos_id = len(dictionary.keys())

    # to index list
    texts = texts.apply(lambda x: dictionary.doc2idx(x, unknown_word_index=eos_id))
    texts = texts.apply(lambda x: np.array([a for a in x if a != eos_id]))
    max_sentence_length = max(texts.apply(len)) if max_sentence_length is None else min(max(texts.apply(len)), max_sentence_length)

    # padding
    texts = texts.apply(lambda x: x[:max_sentence_length])
    texts = texts.apply(lambda x: np.pad(x, (0, max_sentence_length - len(x)), 'constant', constant_values=(0, eos_id)))

    # change types
    texts = texts.apply(lambda x: x.tolist())
    descriptions.id = descriptions.id.astype(np.int32)

    return np.array(descriptions.id.values), np.array(list(texts.values), dtype=np.int32), len(dictionary.keys()) + 1


In [139]:
ids, texts, n_word = make_item_descriptions()

In [141]:
np.array(list(texts), dtype=np.int32)

array([[   35,     6,    11, ..., 38545, 38545, 38545],
       [   35,    57,    61, ..., 38545, 38545, 38545],
       [   35,   128,   118, ..., 38545, 38545, 38545],
       ...,
       [   35, 38490, 38490, ..., 10494,    79, 38137],
       [   35, 38528, 38528, ..., 38545, 38545, 38545],
       [   35, 32051, 38533, ..., 38545, 38545, 38545]], dtype=int32)

In [19]:
ls.describe()

count    2243.000000
mean       51.283103
std        25.754721
min         7.000000
25%        31.000000
50%        49.000000
75%        65.000000
max       174.000000
Name: description, dtype: float64

In [22]:
movie_df = pd.read_csv(os.path.join('data', 'tmdb-5000-movie-dataset', 'tmdb_5000_movies.csv'))

In [24]:
movie_df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [25]:
movie_df['keywords'].head()

0    [{"id": 1463, "name": "culture clash"}, {"id":...
1    [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2    [{"id": 470, "name": "spy"}, {"id": 818, "name...
3    [{"id": 849, "name": "dc comics"}, {"id": 853,...
4    [{"id": 818, "name": "based on novel"}, {"id":...
Name: keywords, dtype: object

In [51]:
credit_df = pd.read_csv(os.path.join('data', 'tmdb-5000-movie-dataset', 'tmdb_5000_credits.csv'))

In [52]:
credit_df = credit_df.head()

In [70]:
import json
credit_df['crew'] = credit_df['crew'].apply(json.loads)
credit_df['crew'].apply(lambda xs: [x['name'] for x in xs[:10]]).values

array([list(['Stephen E. Rivkin', 'Rick Carter', 'Christopher Boyes', 'Christopher Boyes', 'Mali Finn', 'James Horner', 'James Cameron', 'James Cameron', 'James Cameron', 'James Cameron']),
       list(['Dariusz Wolski', 'Gore Verbinski', 'Jerry Bruckheimer', 'Ted Elliott', 'Terry Rossio', 'Stephen E. Rivkin', 'Craig Wood', 'Hans Zimmer', 'Mike Stenson', 'Eric McLeod']),
       list(['Thomas Newman', 'Sam Mendes', 'Anna Pinnock', 'John Logan', 'John Logan', 'Dennis Gassner', 'Ian Fleming', 'Lee Smith', 'Bill Bernstein', 'Daniel Craig']),
       list(['Hans Zimmer', 'Charles Roven', 'Christopher Nolan', 'Christopher Nolan', 'Christopher Nolan', 'Christopher Nolan', 'Jonathan Nolan', 'Emma Thomas', 'Wally Pfister', 'Wally Pfister']),
       list(['Andrew Stanton', 'Andrew Stanton', 'John Lasseter', 'Colin Wilson', 'Gail Stevens', 'Tom Johnson', 'Eric Zumbrunnen', 'Michael Silvers', 'Juan Peralta', 'Paul Gooch'])],
      dtype=object)

In [69]:
credit_df['crew'].values[0][0]

{'credit_id': '52fe48009251416c750aca23',
 'department': 'Editing',
 'gender': 0,
 'id': 1721,
 'job': 'Editor',
 'name': 'Stephen E. Rivkin'}

In [64]:
credit_df.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')